# Teacher Session 12 - Data Transformation


## This session will cover:

- Sorting dataframe
- Groupby on temporal data
- Pivoting/casting/spreading rows into columns (long to wide format)
- Unpivoting/melting/gathering columns into rows (wide to long format)

In [ ]:
import pandas as pd 

## Loading dataset

In [ ]:
## The air_quality_no2_long.csv data set provides 𝑁𝑂2 values for the measurement stations FR04014, BETR801 and London Westminster in respectively Paris, Antwerp and London.
air_quality_no2 = pd.read_csv('https://raw.githubusercontent.com/msh209/INT3625/main/data/air_quality_no2_long.csv'\
                              ,parse_dates=True)

In [ ]:
## The air_quality_pm25_long.csv data set provides 𝑃𝑀25 values for the measurement stations FR04014, BETR801 and London Westminster in respectively Paris, Antwerp and London.
air_quality_pm25 = pd.read_csv('https://raw.githubusercontent.com/msh209/INT3625/main/data/air_quality_pm25_long.csv',\
                               parse_dates=True)

In [ ]:
air_quality_no2
# air_quality_pm25

In [ ]:
## we concatinate no2 and pm25 measures into a single dataset
air_quality = pd.concat([air_quality_pm25, air_quality_no2], axis=0)
air_quality

## Sorting dataframe

#### We can use sort_values to sort a dataframe based on a column or combination of columns:

    df.sort_values(
        by= <Name or list of names to sort by>,
        ascending= <bool or list of bool, default True, Sort ascending vs. descending. Specify list for multiple sort orders. If this is a list of bools, must match the length of the by.>,
        inplace=<If True, perform operation in-place. i.e the dataframe is overwitten with a sorted copy>
    )

Learn more about sorting dataframe here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html

#### The following sorts measurements by country in descending order

In [ ]:
air_quality.sort_values(by='country',ascending=False)
# air_quality.sort_values(by=['country','value'],ascending=False)  #MH added

####  The following sorts measures by parameter then by the value; former in descending order and the latter ascending order

In [ ]:
air_quality.sort_values(by=['parameter','value'],ascending=[False,True])

## Groupby on temporal data

Pandas "to_datetime" converts a scalar, array-like, Series or DataFrame/dict-like to a pandas datetime object.

Read more here: https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

Here we convert date.utc to a pandas datetime object and use utc=True since the datetime is in universal Coordinated Time

In [ ]:
air_quality['date_formatted']=pd.to_datetime(air_quality['date.utc'],utc=True)

In [ ]:
air_quality.head()

We can then use pandas grouper to set the frequency for group by operation on datetime object.

Read more here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html

The following code finds average value of measurements per each day for city, country, location, and parameter.

In [ ]:
air_quality_daily=air_quality.groupby(['city','country','location','parameter','unit',\
                                       pd.Grouper(key='date_formatted',freq='1D')])[['value']].mean().reset_index()
air_quality_daily.head()

## From long to wide Dataframes: Pivot operations 

#### The following figure provides a visual representation of the operation we want to perform
<img src="https://github.com/mosleh-exeter/BEM1025/raw/main/images/session05-fig1.png">



pivot_table creates a spreadsheet-style pivot table as a DataFrame.

The levels in the pivot table will be stored in MultiIndex objects (hierarchical indexes) on the index and columns of the result DataFrame.

    pandas.pivot_table(data, values=None, index=None, columns=None)

    data: DataFrame
    
    values: column to aggregate, optional
    
    index: column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The list can contain any of the other types (except list). Keys to group by on the pivot table index. If an array is passed, it is being used as the same manner as column values.

    columns: column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The list can contain any of the other types (except list). Keys to group by on the pivot table column. If an array is passed, it is being used as the same manner as column values.

Read more: https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

In [ ]:
air_quality.head()

### What about having a dataframe where we have parameters as columns

In [ ]:
air_quality_pivoted=air_quality.pivot_table(index=['date.utc','city','country','location','unit'], 
                                            columns='parameter', 
                                            values='value').reset_index()

In [ ]:
air_quality_pivoted.shape,air_quality_pivoted.shape

In [ ]:
air_quality_pivoted.head()

### What about having a dataframe where we have cities and parameters as columns

In [ ]:
#air_quality_pivoted=
air_quality.pivot_table(
        values='value', 
        index=[ 'date.utc','country','location'], 
        columns=['parameter','city'])

## From wide to long form: Melt operations

### The following figure provides a visual representation of the operation we want to perform
<img src="https://github.com/mosleh-exeter/BEM1025/raw/main/images/session05-fig2.png">

pandas melt unpivots a DataFrame from wide to long format, optionally leaving identifiers set.

This function is useful to massage a DataFrame into a format where one or more columns are identifier variables (id_vars), while all other columns, considered measured variables (value_vars), are “unpivoted” to the row axis, leaving just two non-identifier columns, ‘variable’ and ‘value’.

    pandas.melt(dataframe, id_vars=None, value_vars=None)

    id_vars: tuple, list, or ndarray, optional
            Column(s) to use as identifier variables.
    
    value_vars: tuple, list, or ndarray, optional
            Column(s) to unpivot. If not specified, uses all columns that are not set as id_vars.



### what about transform air_quality_pivoted into the original long format where we have parameter and values

In [ ]:
air_quality_melted=air_quality_pivoted.melt(id_vars=['city','country','date.utc','location'],
                                            value_vars=['no2','pm25'],
                                            var_name='parameter',
                                            value_name='value')

air_quality_pivoted.shape,air_quality_melted.shape

air_quality_melted.head()

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

### See see more examples here: 

https://towardsdatascience.com/reshape-pandas-dataframe-with-pivot-table-in-python-tutorial-and-visualization-2248c2012a31

https://towardsdatascience.com/reshape-pandas-dataframe-with-melt-in-python-tutorial-and-visualization-29ec1450bb02